In [1]:
import numpy as np
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.datasets as datasets
import torchvision.transforms as transforms

from torch.utils.data import DataLoader,TensorDataset

from preprocess import *
from attacks import *
from models import *
from utils import *
from generate_attacks import *

device = 'cuda'
torch.cuda.set_device(8)

In [2]:
train_set = datasets.MNIST(root = '/home/aminul/data',train = True, 
                           transform = transforms.ToTensor(), download = False)
test_set = datasets.MNIST(root = '/home/aminul/data',train = False, 
                          transform = transforms.ToTensor(), download = False)

In [3]:
train_loader = DataLoader(train_set,100,True)
test_loader = DataLoader(test_set,100,False)

train_loader_2 = DataLoader(train_set,1,True)
test_loader_2 = DataLoader(test_set,1,False)

In [4]:
torch.save(train_loader, 'dataloader.pth')

In [5]:
obj = torch.load('dataloader.pth')
obj

In [6]:
num_classes = 10
learning_rate = 0.01
num_epochs = 5

In [7]:
criterion = nn.CrossEntropyLoss()

model1 = CNN()

opt_1 = torch.optim.SGD(model1.parameters(),lr=learning_rate)

In [8]:
history = train(model1,state,num_epochs,opt_1,criterion,device)
test(model1,test_loader,criterion,device)

  1%|          | 6/600 [00:00<00:11, 52.99it/s]

Training: epoch: [1/5]  loss: [2.07] Accuracy [38.47] 


  2%|▏         | 9/600 [00:00<00:07, 82.94it/s]

Training: epoch: [2/5]  loss: [0.50] Accuracy [85.38] 


  1%|          | 5/600 [00:00<00:13, 44.05it/s]

...

Training: epoch: [4/5]  loss: [0.25] Accuracy [92.27] 


  6%|▌         | 6/100 [00:00<00:01, 54.07it/s]

Training: epoch: [5/5]  loss: [0.21] Accuracy [93.81] 


100%|██████████| 100/100 [00:01<00:00, 72.44it/s]

Testing: Loss: [0.17] Accuracy [94.74]


In [13]:
dir = os.getcwd()
file_name = "/model.pth.tar"
path = dir + file_name
print(path)

/home/aminul/Adversarial-Attacks/model.pth.tar


In [16]:
torch.save(model1.state_dict(), path)

In [17]:
model = CNN()
model.load_state_dict(torch.load(path))
model.eval();

CNN(
  (conv1): Conv2d(1, 6, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(6, 12, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=300, out_features=100, bias=True)
  (fc2): Linear(in_features=100, out_features=10, bias=True)
)

In [ ]:
test(model,test_loader,criterion,device)

 94%|█████████▍| 94/100 [00:02<00:00, 45.91it/s]